# MIP Velocity Control

In [ ]:
import sys
sys.path.append('..')

In [ ]:
%matplotlib notebook  
import numpy as np
import matplotlib.pyplot as plt
import math
import time

In [ ]:
from ctrl.system.tf import PID
from ctrl.block.linear import MIMO, ShortCircuit, Subtract, Differentiator, Sum
from ctrl.block.logger import Logger
from ctrl.block.nl import Abs
from ctrl.block.logic import CompareAbs, Trigger
from ctrl.system.tf import DTTF
from ctrl.system.ss import DTSS

# Initialize the server on the beagblebone
python3 server.py -H 192.168.12.197 -m mip3 -t 0.01

Substitute `HOST` and `PORT` with the IP address and port of your controller:

In [ ]:
from ctrl.client import Controller
HOST, PORT = "localhost", 9999
#HOST, PORT = "192.168.12.197", 9999
mip = Controller(host = HOST, port = PORT)

## Probe the controller

In [ ]:
print(mip.info('all'))

## Complementary filter for estimating $\theta$

In [ ]:
Ts = 0.01
fc = 1
l = 2 * math.pi * fc
a = math.exp(-l*Ts)

Af = np.array([[a]])
Bf = np.array([[1-a,(1-a)/l]])
Cf = np.array([[1]])
Df = np.array([[0,0]])

fltr = DTSS(Af,Bf,Cf,Df)

mip.add_signal('theta_fltr')
mip.add_filter('fltr', 
               MIMO(fltr),
               ['theta','theta_dot'], 
               ['theta_fltr'])

## Calculate encoder velocity

In [ ]:
# phi is the average of the encoders
mip.add_signal('phi')
mip.add_filter('phi', 
               Sum(gain=0.5),
               ['encoder1','encoder2'], 
               ['phi'])

# Differentiate phi
mip.add_signal('phi_dot')
mip.add_filter('phi_dot', 
               Differentiator(),
               ['clock','phi'], 
               ['phi_dot'])

## Calculate output errors

In [ ]:
mip.add_signal('theta_dot_reference')
mip.add_signal('phi_dot_reference')
mip.add_signal('voltage')

mip.set_signal('theta_dot_reference', 0)
mip.set_signal('phi_dot_reference',0)

mip.add_signal('theta_dot_error')
mip.add_signal('phi_dot_error')

mip.add_filter('sub_theta', 
               Subtract(),
               ['theta_dot','theta_dot_reference'], 
               ['theta_dot_error'])

mip.add_filter('sub_phi', 
               Subtract(),
               ['phi_dot','phi_dot_reference'], 
               ['phi_dot_error'])

## Setup controller

In [ ]:
#0.01
Ac = np.array([[0.913134, 0.0363383],[-0.0692862, 0.994003]])
Bc = 2*np.pi*np.array([[0.00284353, -0.000539063], [0.00162443, -0.00128745]])
Cc = np.array([[-383.009, 303.07]])
Dc = 2*np.pi*np.array([[-1.22015, 0]])

Bc = (100/7.4)*Bc
Dc = (100/7.4)*Dc

K = 1

ctrl = DTSS(Ac,K*Bc,Cc,K*Dc)

In [ ]:
mip.add_filter('controller', 
               MIMO(ctrl),
               ['theta_dot_error','phi_dot_error'], 
               ['voltage'])

In [ ]:
mip.add_filter('cl1',
                ShortCircuit(),
               ['voltage'],
               ['motor1'])
mip.add_filter('cl2',
                ShortCircuit(),
               ['voltage'],
               ['motor2'])

#mip.remove_filter('cl1')
#mip.remove_filter('cl2')

In [ ]:
mip.add_sink('logger', 
             Logger(), 
             ['clock','duty',
              'theta','theta_fltr',
              'theta_dot','encoder1','encoder2',
              'phi','phi_dot',
              'theta_dot_error','phi_dot_error',
              'voltage',
              'motor1','motor2'])

In [ ]:
print(mip.info('all'))

In [ ]:
from IPython.display import display
import ipywidgets as widgets

def set_reference(value):
    print('setting phi_dot_reference to {}'.format(value))
    mip.set_signal('phi_dot_reference',value/360)
    
w = widgets.interactive(set_reference, value=(-100,100))
display(w)

In [ ]:
mip.set_sink('logger',reset=True)
mip.set_source('clock',reset=True)
mip.set_source('encoder1',reset=True)
mip.set_source('encoder2',reset=True)
mip.set_filter('controller',reset=True)
mip.set_source('imu',reset=True)
with mip:
    time.sleep(120)
log = mip.read_sink('logger')

In [ ]:
clock = log[:,0]           # 'clock',
duty = log[:,1]            # 'duty'
theta = log[:,2]           # 'theta'
theta_fltr = log[:,3]      # 'theta_fltr'
theta_dot = log[:,4]       # 'theta_dot'
encoder1 = log[:,5]        # 'encoder1'
encoder2 = log[:,6]        # 'encoder2',
phi = log[:,7]             # 'phi'
phi_dot = log[:,8]         # 'phi_dot'
theta_dot_error = log[:,9] # 'theta_dot_error'
phi_dot_error = log[:,10]   # phi_dot_error',
voltage = log[:,11]        # voltage
motor1 = log[:,12]         # motor1
motor2 = log[:,13]         # motor2

data = {
    'clock': clock,
    'theta': theta,
    'theta_dot': theta_dot,
    'phi_dot': phi_dot,
    'encoder1': encoder1,
    'encoder2': encoder2,
    'voltage': voltage
}

In [ ]:
fig = plt.figure()

#plt.plot(clock, imu, clock, encoder1, clock, encoder2, clock, voltage)
plt.plot(clock[1:],clock[1:]-clock[:-1], '.',clock,duty,'.')
plt.xlabel('t (s)')
plt.ylabel('position (cycles)')
plt.grid()
#plt.xlim(0,1)

In [ ]:
fig = plt.figure()

#plt.plot(clock, imu, clock, encoder1, clock, encoder2, clock, voltage)
plt.plot(clock,theta_dot*360,'b.')
plt.xlabel('t (s)')
plt.ylabel('position (cycles)')
plt.grid()
#plt.xlim(0,2.45)
#plt.xlim(0,3)

In [ ]:
fig = plt.figure()

#plt.plot(clock, imu, clock, encoder1, clock, encoder2, clock, voltage)
plt.plot(clock,encoder1*360,'r.',clock,encoder2*360,'b.',clock,phi*360,'g.')
plt.xlabel('t (s)')
plt.ylabel('velocity (deg/s)')
plt.grid()
#plt.xlim(1.8,2.45)
#plt.ylim(-60,65)

In [ ]:
fig = plt.figure()

#plt.plot(clock, imu, clock, encoder1, clock, encoder2, clock, voltage)
plt.plot(clock,phi_dot*360,'r.-')
plt.xlabel('t (s)')
plt.ylabel('velocity (deg/s)')
plt.grid()
#plt.xlim(1.8,2.45)
#plt.ylim(-60,65)

In [ ]:
fig = plt.figure()

#plt.plot(clock, imu, clock, encoder1, clock, encoder2, clock, voltage)
plt.plot(clock,(7.4/100)*voltage,'b.')
plt.xlabel('t (s)')
plt.ylabel('voltage (V)')
plt.grid()
plt.ylim(-7.4,7.4)

In [ ]:
fig = plt.figure()

#plt.plot(clock, imu, clock, encoder1, clock, encoder2, clock, voltage)
plt.plot(clock,theta_fltr*360,'b.-')
plt.xlabel('t (s)')
plt.ylabel('theta (deg)')
plt.grid()
#plt.xlim(1.8,2.45)
#plt.ylim(-60,65)

In [ ]:
import scipy.io as spio
spio.savemat(file_name='~/Dropbox/ZhuZhuo/python', mdict = data)